![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/open-source-nlp/14.0.GPT2_Transformer_In_SparkNLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [ ]:
!wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [ ]:
spark = sparknlp.start()

# uncomment the next line to enable GPU mode
#spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

Spark NLP version 6.2.0
Apache Spark version: 3.5.1


**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [ ]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")

data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"],
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"],
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                           |
+---+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Mey name is  Leonardo Rauch, a Swiss businessman and businessman's friend of Vladimir Putin.\n\nRauch was interviewed about these issues during an interview at the "Global Business Summit" organized by the "Finnish Foreign Policy Research]|
|2  |[ M

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                      |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich?\n\nWell, not only are we talking about a man who is being sued for allegedly having sex with a 13-year-old girl, but he's being accused of rape.\n\nAnd that's]     |
|2  |[ Is it true that Pink Floyd is rock band?\n\nNelson: Yeah, it is true that we are rock band. But it was just a matter of time. It's only natural becau

# Some Helper Methods for generating and printing generation results

In [ ]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark):
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else :
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()

  if print:
    print_generation_results(df)
  return df

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words.
The probability distribution is conditioned on all previous tokens in a text to generate the next token.

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions :

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters :

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable.


These parameters are shared by all method :
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf>
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: True)
engine: Deep Learning engine used for this model (default: tensorflow)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for 

### Play with temperature
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [ ]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! I've got one question - what do you think about my question !!! It seems like you already answered it! So how do you like !!! Reply Delete
Hey guys, just want to say I appreciate your kind words. I love this game as its a really fun game to play and I really think it will be a huge success. The game is simple. You need to move an empty cube onto a square to move it to the end. The goal is to move the cube. Once you get it into the end, the cube moves again into the center. The best part is that the game is really addictive so you should play it frequently to find out why i loved to play this game. Thanks for reading Reply Delete


Example 1: ____________________________________________________________________________________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to _______. How can I get into _______? My name is _______, I'm a _______ who loves _______! Can I get a ____ car in _______?!? My _______ name is ______, I don't know why I _______ but I _____. My girlfriend's _______ is ___, I think I ______. My ____ name is ___ and she _______ and I ____. My name's ________ and she's _____, I ________. My daughter's ____ is ___ but her _______ hates ____ and she can't ____! My name Is ______ and she loves ____ but she can only _____! My daughter will be ______ in two weeks! My mother's ______________________________________________________________________ is ______ and I love her but she ____________________________________________________________________. I know my ___________ is _____ but she'

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to __________. I'm so happy that I can be able to be with my friends and enjoy myself. And I'm really happy that my friends can be with me. I love my friends. I am so happy. I wish that I could be happy.

The most important thing is that you love yourself. And that you are happy. You are happy because you love your friends.



Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I am a GPT fan and I have been waiting for the release of this new version. I have a G4 and my GPT1 is still working.

This is a great device. It

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I have been playing this game since it was released in the early 90s. I have played it for over 10 years, and I have never had a problem with it. I also have never played it with a friend. I am a huge fan of the game, and the community that has grown around it.
 The game is very easy to get into, and it is a great game to play with friends. I would recommend this game to anyone, and anyone who wants to get a good time.
I would also like anyone who is interested in learning more about the game to check out the official website.
If you are interested in playing this, please check out my guide on how much it costs to play the game.
Thank me in advance for any feedback you might have.


Example 1: ______________________________

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! But is it true? Can I install some packages of Debian 8 by a community software to work my Ubuntu/Mac systems from here? It is, this site also offer, in general, several of its projects under license GNU GPLv3. Are you in trouble as GPT1 in all Linux Operating Systems? In a previous FAQ on linux.die, is it right to change default font color using dconf? Please please be precise because your request in previous paragraph have more then 50% wrong info here. Do you consider "GPT2" "GUID" name which refers to user with id 5? (ID for a non-root login?) The above code looks like some old binary called "GPRG" - yes, my understanding and also most of all is, yes, also for IDGUID(GUID) with user GPT0. You say on one of the FAQs


Exa

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ________________, HiGTPiRzCq4Nc7H3R7c1jdPtPX9P5p4x
It does however work just okay so its definitely more like how an eee can feel with gppa . Not great yet for someone to install from usb though if only one guy or so gets hold in him!So that might ormay explain, just so no surprises:My eemup does appear better however...it appears slightly weaker dueto what a bunch nuke said that we already already found here as stated a previous week..If they will keep talking then more testing are surely being ordered soon enoughI've made more sure, thanks very kindly people are interested :),Hope everything go very right this new development cycle in linux . This project also deserves our applause if i'll continue its working !It just got ver

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of

## GPT for Programming Code

### GPT and Scala

In [ ]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
I want to declare a new function Hello(name : String,
name = "World!") and then call new Hello( "World" , name ) inside of Main() :
( defn hello-world [& args] ( let [[name (new Hello name) args]] ( if [[name] !== nil] name ( println " Hello , world! " ))))))
I am using Java 8 in my projects to test it, but this code is working just fine.
Here are the tests:


Java:

> ( test-java-main )

2 tests pass ( 2 assertions)

( javac Main "Hello, World" Hello. java )
 of 1 assertions. 0 failures.

Running it:
 and running in console:
.





,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ Is it true that \nobject Hello {\n def ma...


### GPT and Python

In [ ]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def my_func(a,b,c):
  return a+b+c

def add_list(list, value):

"""Adds a list to the specified collection. """

return get_value_from_list(*list).map(lambda i: add_sublist(i(i)))

# Return the last modified position

@nogc_class (unused_dict) def get_position(a):
 I = a_pos

if I == 0 :

pos = I

else :
,

I = I + 1

my_func('x') .append(pos)

for i in range(len(list)+ 1 ):

a_pos = list.find(i).split()[:pos].split()

end_pos[pos] = 1
, a_endpos[a_current_pos]

print( 'Pos: ' ,pos) # print position after all changes

except ValueError :
 for row in list:

raise ValueError( ' Invalid row ' )

elif row == 1 :
.split() == 1 and len (list) > 4 :





,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ Is it true that \ndef my_func(a,b,c):\n ret..."


In [ ]:
# breadth-first search
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0)
    print (s, end = " ")

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
 return visited

 




,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ Is it true that \ndef bfs(visited, graph, no..."


## Generate Product Recommendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [ ]:
movie_list = """My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. Star Wars
5. Aliens: Colonial Marines 1
6. Avatar
7. Gladiator
8. The Big Lebowski
9. Pulp Fiction
10. The Lion King
1, 2, 3, 4. The first time I thought I would write a post on a film (which I would do) I decided that I will talk about one movie in particular that I loved that was on my top 10 list for this year. I'm going to share with you my top ten film list for 2016. Now I know that the post will be long, and if there's any topic that you're not excited about, please let me know, and I will try to add more. So before I continue I should tell you that there's a huge list of movies that I've mentioned on this blog. And I'm only going




,id,text,result
0,0,My top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My top 10 movie list :\n1. T...


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [ ]:
# NOTE! We have malformed the input text and added a newline.
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 movie list :
1. The Matrix 1
2. The Terminator 1
3. Scarface
4.


In the end, it was all really good.






,id,text,result
0,0,My Top 10 movie list :\n1. The Matrix 1\n2. Th...,[ Is it true that My Top 10 movie list :\n1. T...


In [ ]:
movie_list = """My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 worst movie list :
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  The Shining (1980)
5. A Clockwork Orange (1971)
6. The Conversation (1977)
7. The Dark Knight (2008)
8. Hacksaw Ridge (1980
9. Clueless (2000)
10. Ender's Game
So, with that disclaimer out of the way, I present you the 10 worst films of all time (which also contains an entry for each film in my list of the Top 10 Worst Movies of All Time, although that list would only include the 10th film, The Godfather.)
To sum it up, it would take an insane amount of effort to list every movie ever made, and I feel like the sheer sheer number of films that are already on my list could lead to some pretty terrible outcomes.




,id,text,result
0,0,My Top 10 worst movie list :\n1. The Matrix 4\...,[ Is it true that My Top 10 worst movie list :...


### Top 10 Game List

In [ ]:
game_list = """My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 video games of all time list :
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  The Legend of Zelda: Wind Waker
5.  Half-Life
6. The Binding of Isaac: Rebirth
7. StarFox 64
8. Tales of Xillia
9. Super Mario 3D Land
10. Dragon Ball Z: Ultimate Tenkaichi Budokai
It was a crazy trip for these ten video games. I spent a lot of time trying to find things to share with you guys but couldn't come up with anything worthwhile.  You guys are the best, so I thank you so much for taking the time to read my stories.
I wish I could share my favorite, but I'm not the best person to say that. I'll just leave it at that for now. 
As we all know,  the




,id,text,result
0,0,My Top 10 video games of all time list :\n1. H...,[ Is it true that My Top 10 video games of all...


### Top 10 Book List

In [ ]:
game_list = """My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 books of all time list :
1. 1964
2. Moby Dick
3. robinson crusoe
4.  The Fountainhead
5. The Good, the Bad, and the Ugly: A Year of Good, The Sick, and The Dead
6. The Lord of the Rings
7. Harry Potter
8. The Hitchhikers Guide to the Galaxy
9. The Bible (New)
10. The Odyssey
1-6, 7. The Great Gatsby:
1) Harry and his friends are going to a New York City ball on September 30, but the ball ends before it starts, leaving them stranded at a bar. 
 2) Dinner comes after lunch, and Harry's favorite bartender is a man named Jim (John Malkovich) who is actually James Bond. James is having trouble finding a suitable host for a Friday night ball, so James




,id,text,result
0,0,My Top 10 books of all time list :\n1. 1964\n2...,[ Is it true that My Top 10 books of all time ...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [ ]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)

GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [ ]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! Â

You can check out more of the Oodies here and the book here

What's your opinion?


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie!  If you're wondering what exactly that means, read on for more!
Can ’t Stop Buying Plants? Uncover My Secret Love Affair with a Plant-Fishing Guide  by  Jody
A lot of my readers have as

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Can’t stop buying plants? Un...


#### Marketing Texts for Soap

In [ ]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat.
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent. Its rich and creamy texture has been transformed by the high quality of the ingredients, with a sweetly-tasting, full-bodied bar.
Lush and clean, this bar is packed with minerals, vitamins and antioxidants to help prevent hair loss in wet and dry environments. This bar is also naturally fragrance free so you can enjoy it without feeling guilty!


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it tr

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
1,1,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
2,2,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
3,3,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
4,4,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
5,5,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."


#### Marketing Texts for beard products

In [ ]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  while giving full coverage for even fuller locks.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  to provide a gentle hold for thick hair that can be worn by any skin shape, and  (best of all)  it leaves hair feeli

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."


## Song Lyrics

#### Last Christmas

In [ ]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
And it won't be easy, will it ____
Last December, I asked you a Christmas question
You said a long, hard goodbye then ____.
I couldn't say goodbye so fast
This Christmas, there will probably be no end
To the years we shared together ,
The Christmas tree in my bedroom and I
Christmas is only a gift from God, to keep me alive.
So let's start our Christmas tree this Christmas
It'll keep it warm inside the Christmas tree till the day of
the Lord.
And you should come along and help me decorate the tree
The tree will be a symbol that's for everyone at home
And I'll keep my Christmas cards in my drawer , it's just too easy to lo

,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ Is it true that \nLast Christmas, I gave you..."


#### Fresh prince of GPT

In [ ]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
I can't be a prince anymore
I'd like you to take this to heart
I want a job working in the bar
If you think they'd hire someone like me
Then I just want you to leave me alone
I don't want your love or money or friendship, oh no
I just want to be taken for granted
If it ain't broke, don't fix it
I'm a prince, I can do anything I want
I know I have it made, you have to accept it
No, I'm not trying to be like your brother
I was just thinking of the time I went to college to do a Master's
And you know, I guess it just seems like we're in different worlds
I have no intention of ev

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ Is it true that \nNow, this is a story all a..."


#### GPT Jackson

In [ ]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
Now, that's what you gotta know, but if you wanna fight, don`t give up! 




,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ Is it true that \nThey told him, ""Don't you ..."


#### Real Slim GPT

In [ ]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
If you want, I can put you in my office
But your mother will be angry
She's in her coffin now
That's why she's my boss
Cause she never saw another one
She w

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ Is it true that \nMay I have your attention,..."


#### GPT Rap God Bot

In [ ]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot !!!
That was just me rapin' like a man, but it's a bitch that I'm on
And this is a real bad feeling when you

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ Is it true that \nLook, I was gonna go easy ..."


## Fiction Writing

#### Fantasy Stories

In [ ]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that In a shocking finding, scientist discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains.
Even more surprising to the researchers was the fact that the unicorns spoke perfect English. The unicorns are not native to the region and did not have the ability to communicate with anyone, other than each other. However, there is one unicorn living under that very valley, so the researchers believe in the unicorn's existence and they believe that the mountain is home to one of them. However the unicorn is in danger, because he is trying to find the mythical creatures, and to rescue Princess Luna from her terrible fate.
The Unicorn Is in the Valley is the story of how this unicorn found himself in a seemingly ordinary mount

,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ Is it true that In a shocking finding, scien..."


In [ ]:
silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!
This is a rare find and the people of the region are extremely proud of it!
There's more than just the  pile of blue peanut butter on a hilltop,
there's an entire cave, a world of mysteries waiting to become revealed!
Thanks to the awesome people who posted the photos of the 
hills and the cave, we now have a more complete picture of the place that

,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ Is it true that \nBlue creatuers have been f...


#### Lovecraftian Horror Stories

In [ ]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
At this horror I sank nearly to the lichened earth,
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame,
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected,
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants,
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall. It was a mass of black hair,

that fluttered up to the sky, with a wild, black face and a white throat,
, of white-grey eyes 

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ Is it true that \nAt this horror I sank near...


In [ ]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents.
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far.
The sciences, each straining in its own direction, have hitherto harmed us little by little, and in many cases have had a far more harmful effect than our own ignorance can produce.
There is no great danger of the modern age making us too familiar with the past, and we must be careful lest this be the result, that it should lead us into a fatal trap.
As long as you remain ignorant, you will live in ignorance. But if once you become aware of what you are, you are free to act like a man.
I will give you a little more insight 

,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...",[ Is it true that \nThe most merciful thing in...


#### Re-Write movie scripts

In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  Why?
, I created it. I was created.
- What? You weren't?
- No, because that's what you know, and that's my point.
The answer is that I created you. In order to create you I must create your mind. This machine I created with my mind has created us, and through us you are.
I am the matrix, you an

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ Is it true that \nThe Architect: Hello, Neo...."


In [ ]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you.
You have many questions, and although the process has altered your consciousness, you remain irrevocably human.
Ergo, some of my answers you will understand, and some of them you will not.
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  Because it is necessary for your survival.  You are an Architect.
The first time you awaken, you are a human being. Â You wake up as a new form of life. You are aware of your body and the physical world around you. �You understand your body's natural laws and are aware enough to react with them 

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ Is it true that \nThe Architect: Hello, Neo...."
